In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import yfinance as yf
import seaborn as sns

In [2]:
"""Sobrescrevendo os métodos do pandas_datareader"""
yf.pdr_override()

In [3]:
tickers = "^BVSP ^GSPC GC=F USDBRL=X"
carteira = yf.download(tickers, start="2010-01-01")["Close"]

[*********************100%***********************]  4 of 4 completed


In [7]:
carteira.columns = ["OURO", "DOLAR", "IBOV", "S&P500"]
carteira = carteira.dropna()
carteira["OURO_BRL"]=carteira["OURO"]*carteira["DOLAR"]
carteira["S&P500_BRL"]=carteira["S&P500"]*carteira["DOLAR"]

In [8]:
#retornos = carteira.pct_change().dropna()
retornos = carteira.pct_change()[1:]

In [9]:
retornos

,OURO,DOLAR,IBOV,S&P500,OURO_BRL,S&P500_BRL
Date,,,,,,
2010-01-05,0.000358,0.010471,0.002784,0.003116,0.010833,0.013620
2010-01-06,0.015920,-0.003166,0.006962,0.000546,0.012703,-0.002623
2010-01-07,-0.002465,0.004274,-0.003930,0.004001,0.001798,0.008292
2010-01-08,0.004501,-0.003968,-0.002669,0.002882,0.000515,-0.001098
2010-01-11,0.010982,-0.000577,0.002419,0.001747,0.010399,0.001168
...,...,...,...,...,...,...
2022-05-05,0.003749,-0.007926,-0.028059,-0.035650,-0.004207,-0.043294
2022-05-06,0.003842,0.022201,-0.001605,-0.005674,0.026128,0.016400
2022-05-09,-0.012811,0.010422,-0.017929,-0.032037,-0.002523,-0.021949


In [24]:
desvio = retornos - retornos.mean()
desvio_quadrado = desvio**2
media_desvio_quadrado = desvio_quadrado.mean()

volatilidade = np.sqrt(media_desvio_quadrado)
volatilidade

OURO          0.010459
DOLAR         0.011370
IBOV          0.015715
S&P500        0.011144
OURO_BRL      0.015154
S&P500_BRL    0.015670
dtype: float64

In [25]:
retornos.std()

OURO          0.010461
DOLAR         0.011372
IBOV          0.015717
S&P500        0.011146
OURO_BRL      0.015157
S&P500_BRL    0.015672
dtype: float64

Por que eles não combinam? Porque, por padrão, o método .std() calcula o desvio padrão da amostra, o que significa que ele usa o denominador de 𝑛−1 . Por outro lado, calculamos o desvio padrão da população, que usa um numerador de 𝑛 . Como os retornos observados são considerados amostras observadas de uma distribuição, provavelmente é mais preciso usar o denominador de 𝑛−1 , então vamos refazer nosso cálculo para ver se obtemos o mesmo número.

Para obter o número de observações, podemos usar o atributo .shape de um DataFrame que retorna uma tupla do número de linhas e colunas.

In [27]:
numero_obs = retornos.shape[0]
media_desvio_quadrado = desvio_quadrado.sum()/(numero_obs-1)
volatilidade = np.sqrt(media_desvio_quadrado)
volatilidade

OURO          0.010461
DOLAR         0.011372
IBOV          0.015717
S&P500        0.011146
OURO_BRL      0.015157
S&P500_BRL    0.015672
dtype: float64

# Volatilidade anualizada

Anualizamos a volatilidade escalando (multiplicando-a) pela raiz quadrada do número de períodos por observação

Portanto, para anualizar a volatilidade de uma série mensal, multiplicamos pela raiz quadrada de 12. Em vez de usar o `np.sqrt()`, podemos elevá-lo à potência de $0,5$

Para anualizar uma série diária, devido a variação de dias úteis por ano, podemos definir como valor padrão 240 dias (20 dias úteis mês).

In [40]:
anualizada_vol = retornos.std()*(240**0.5)
anualizada_vol

OURO          0.162059
DOLAR         0.176173
IBOV          0.243493
S&P500        0.172667
OURO_BRL      0.234809
S&P500_BRL    0.242794
dtype: float64

Agora podemos calcular os retornos mensalizados da seguinte forma:

In [49]:
n_dias = retornos.shape[0]
retorno_por_dia = (retornos+1).prod()**(1/n_dias) - 1
retorno_por_dia

OURO          0.000170
DOLAR         0.000368
IBOV          0.000134
S&P500        0.000419
OURO_BRL      0.000538
S&P500_BRL    0.000787
dtype: float64

In [50]:
retorno_mensal = (retorno_por_dia + 1)**20-1
retorno_mensal

OURO          0.003410
DOLAR         0.007389
IBOV          0.002690
S&P500        0.008417
OURO_BRL      0.010825
S&P500_BRL    0.015868
dtype: float64

In [51]:
retorno_anual = (retorno_mensal + 1)**12-1
retorno_anual

OURO          0.041701
DOLAR         0.092363
IBOV          0.032762
S&P500        0.105812
OURO_BRL      0.137916
S&P500_BRL    0.207948
dtype: float64

In [52]:
retorno_anual/anualizada_vol

OURO          0.257320
DOLAR         0.524274
IBOV          0.134549
S&P500        0.612809
OURO_BRL      0.587353
S&P500_BRL    0.856482
dtype: float64

In [57]:
taxa_semrisco = 0.12
retorno_excesso = retorno_anual - taxa_semrisco
sharpe_ratio = retorno_excesso/anualizada_vol
sharpe_ratio

OURO         -0.483153
DOLAR        -0.156874
IBOV         -0.358278
S&P500       -0.082169
OURO_BRL      0.076299
S&P500_BRL    0.362235
dtype: float64